# **Trabalho Prático: Guia de Viagem Inteligente com Roteamento de Cadeias**

**Disciplina:** Inteligência artificial  
**Professor:** Luiz Gustavo  
**Aluno:** Anderson Paes Gomes  

Este notebook implementa a solução para o trabalho proposto que deve atender os requisitos abaixo:

## 1. Objetivo do Projeto

O principal objetivo é desenvolver um sistema que, a partir de uma consulta de um turista, seja capaz de **classificar a intenção da pergunta** e **direcioná-la para a cadeia de processamento mais adequada**. Isso permite a criação de roteiros de viagem personalizados e a resposta a perguntas específicas de forma rápida e eficiente. O sistema deve demonstrar as seguintes vantagens:

- **Roteiros personalizados:** Gerar sugestões de itinerário com base no perfil do turista (ex: aventura, cultural, gastronômico).

- **Módulos especializados por função:** Utilizar cadeias de processamento dedicadas a diferentes tipos de consulta (ex: informações sobre locais, logística de transporte, detalhes de roteiro). **Atenção:** Neste projeto, **não serão utilizados Agentes** no sentido da classe Agent do LangChain, mas sim **Cadeias (Chains)** que realizam funções específicas, orquestradas por um roteador.

- **Geração rápida e automatizada:** Otimizar a velocidade de resposta utilizando um modelo de inferência de alto desempenho como o Groq.

- **Estrutura escalável e personalizável:** A arquitetura modular deve permitir a fácil adição de novas funcionalidades ou bases de conhecimento.

## 2. Tecnologias Envolvidas

Para a realização deste trabalho, vocês deverão utilizar as seguintes ferramentas e conceitos:

- **LangChain:** Como framework principal para orquestrar as cadeias de processamento e a lógica de roteamento.

- **Groq:** Otimizado para alta velocidade, será o motor de inferência para o Large Language Model (LLM) que irá gerar as respostas e os roteiros.

- **Router Chain (LangChain):** O componente central do projeto. Ele será responsável por analisar a consulta do usuário e decidir qual cadeia especializada deve processá-la.

- **Pinecone:** Servirá como base de dados vetorial para implementar o **RAG**. Vocês deverão popular o Pinecone com informações sobre destinos turísticos, pontos de interesse, restaurantes e eventos locais.

- **RAG (Retrieval-Augmented Generation):** Essencial para que o sistema possa acessar informações atualizadas e específicas que não estão no conhecimento pré-treinado do LLM, garantindo a precisão das respostas.

## 3. Estrutura do Projeto

O sistema deverá ser construído em módulos lógicos:

- **Módulo de Entrada:** Recebe a consulta do usuário. A consulta deve incluir o tipo de viagem desejada (ex: "roteiro cultural em Paris por 3 dias", "como chegar ao Coliseu?", "quais são os melhores restaurantes veganos em Tóquio?").

- **Router Chain:** Classifica a intenção do usuário. As classificações podem ser, por exemplo:

    - **roteiro-viagem**
    - **logistica-transporte**
    - **info-local**
    - **traducao-idiomas**

- **Cadeias Especializadas:**

    - **Itinerary Chain (roteiro-viagem):** Recebe o perfil do turista (se disponível), utiliza o RAG para buscar informações sobre atrações e eventos e gera um roteiro detalhado.
    - **Logistics Chain (logistica-transporte):** Responde a perguntas sobre transporte, acomodação e outros aspectos práticos da viagem.
    - **Local Info Chain (info-local):** Fornece informações específicas sobre pontos turísticos, restaurantes, horários de funcionamento, etc., usando o RAG.
    - **Translation Chain (traducao-idiomas): (Bônus)** Esta cadeia implementará um guia de tradução, fornecendo frases úteis para a viagem com base na solicitação do usuário.

- **Base de Conhecimento (RAG):** Crie um conjunto de dados de exemplo sobre uma ou duas cidades turísticas (ex: Rio de Janeiro e Paris) para ser indexado no Pinecone. As informações devem ser relevantes para o turismo (pontos turísticos, restaurantes, dicas de segurança, etc.).

## 5. Requisitos e Entrega

- **Código-fonte:** Um repositório no GitHub contendo todo o código do projeto. O código deve ser bem comentado e organizado.

- **Demonstração:** Um vídeo de 1 a 3 minutos mostrando a funcionalidade do sistema em ação, demonstrando os diferentes fluxos de roteamento e a capacidade de RAG.

### Importar bibliotecas

In [1]:
from typing import List, Dict, Any, Optional
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from dotenv import find_dotenv, load_dotenv
import os
import json
import re
import time

c:\Projetos\Langchain\venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


**Configuração das Chaves de API**

In [2]:
load_dotenv(find_dotenv())

True

## Carregando a LLM da Groq

In [3]:
llm = ChatGroq(model_name='llama-3.3-70b-versatile',temperature=0)

## Carregamento e Processamento dos Documentos PDF

In [4]:
folder_path = 'guia'
filename = 'guia_viagens.pdf'

### Leitura do Documento PDF

In [5]:
document = []
file_path = os.path.join(folder_path, filename)
loader = PyPDFLoader(file_path)
document.extend(loader.load())

## Geração de Embeddings e Armazenamento no Pinecone

### Divisão dos Documentos em Chunks

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(document)

In [7]:
print(f'Total de chunks: {len(docs)}')

Total de chunks: 28


In [8]:
docs[1]

Document(metadata={'producer': 'LibreOffice 25.2.3.2 (X86_64) / LibreOffice Community', 'creator': 'PyPDF', 'creationdate': '2025-09-09T18:32:15-07:00', 'title': 'Guia de Viagens', 'source': 'guia\\guia_viagens.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='linhas (Azul, Vermelha, Verde e Amarela) e conecta bairros importantes. Elétricos \n(bondes) como a Linha 28 oferecem trajetos panorâmicos. Ônibus e trens regionais \ncomplementam a rede. Táxis e aplicativos de transporte são amplamente disponíveis.\nAcomodação\nPrincipais áreas para hospedagem incluem Baixa (central e histórica), Chiado (bairro \nelegante com lojas e cafés), Alfama (mais antigo, com ruas estreitas e ambiente \ntradicional) e Bairro Alto (conhecido pela vida noturna). Cascais e Estoril são opções \nlitorâneas próximas.\nPrincipais Atrações\n• Torre de Belém: Fortificação do século XVI em estilo manuelino às margens do \nTejo; oferece vista panorâmica e detalhes ornamentais como esculturas de \

### Inicializar modelo de embeddings (HuggingFace para uso local/gratuito)

In [9]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\Administrador\AppData\Local\Temp\ipykernel_6444\3701169180.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


### Inicializar Pinecone

In [10]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [11]:
vectorstore = None
retriever = None

In [12]:
index_name = "rag-viagem" 
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()] # change if desired

### Deletar e recriar o índice para garantir que a dimensão esteja correta

In [13]:
if index_name in existing_indexes:
    print(f"Deletando o índice existente '{index_name}'...")
    pc.delete_index(index_name)
    time.sleep(1) # Aguardar a exclusão

Deletando o índice existente 'rag-viagem'...


In [14]:
pc.create_index(
    name=index_name,
    metric="cosine",
    dimension=384, # Isso irá retornar 384
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
while not pc.describe_index(index_name).status["ready"]:
    print("Aguardando o índice ficar pronto..."+pc.describe_index(index_name).status)
    time.sleep(1)

index = pc.Index(index_name)

### Criar ou conectar ao VectorStore do Pinecone

In [15]:
vectorstore = PineconeVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    index_name=index_name,
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

## RouterChain


### Prompt templates para as cadeias
Prompts específicos por intenção, com **RAG** quando aplicável.

In [16]:
base_system = (
    "Você é um assistente de viagens que responde em português do Brasil, com tom claro e útil. "
    "Seja conciso, mas completo. Cite fatos apenas quando aparecerem no contexto."
)

prompt_intinerary = ChatPromptTemplate.from_messages([
    ("system", base_system + " Gere um roteiro detalhado, organizado por dias e períodos. "
     "Considere perfil do viajante (cultural, gastronômico, aventura), duração, cidade e preferências."),
    ("system", "Contexto recuperado (RAG):\n{context}"),
    ("human", "Gere um roteiro para: {query}. Inclua dicas práticas e alternativas em caso de chuva.")
])

prompt_logistics = ChatPromptTemplate.from_messages([
    ("system", base_system + " Responda sobre transporte, acomodação, deslocamentos e custos aproximados quando possível."),
    ("system", "Contexto recuperado (RAG):\n{context}"),
    ("human", "{query}")
])

prompt_localinfo = ChatPromptTemplate.from_messages([
    ("system", base_system + " Forneça informações específicas sobre atrações, horários, restaurantes e eventos."),
    ("system", "Contexto recuperado (RAG):\n{context}"),
    ("human", "{query}")
])

propmpt_translation = ChatPromptTemplate.from_messages([
    ("system", base_system + " Você é um guia de tradução para viagens. "
     "Forneça frases úteis traduzidas e transliterações quando cabível."),
    ("human", "{query}")
])

### Cadeias especializadas
Cada cadeia recebe `{query}` e (quando aplicável) `context` vindo do **retriever**.

In [17]:
# Mensagem padrão caso não haja dados suficientes no RAG
mensagem_falta_dado = (
    "Desculpe, não encontrei essa informação na minha base de dados vetorizada (RAG). "
    "Sugiro realizar uma **nova atualização do RAG** com documentos que contenham esse conteúdo."
)

# Instruções do sistema para limitar respostas ao contexto
system_instrucao_pt = (
    "Você é um assistente que **somente** pode responder com base no CONTEXTO fornecido abaixo.\n"
    "Se a resposta não estiver no contexto, diga explicitamente que **não consta na base de dados** "
    "e **sugira uma atualização do RAG**. Não invente detalhes fora do contexto.\n\n"
    "Regras:\n"
    "1) Use apenas fatos presentes no CONTEXTO.\n"
    "2) Se faltar evidência suficiente, responda com a mensagem padrão de ausência.\n"
    "3) Seja conciso e cite trechos do contexto quando útil.\n"
)

# Template da mensagem do usuário com contexto
user_template_pt = (
    "PERGUNTA:\n{question}\n\n"
    "CONTEXTO (trechos do RAG):\n{context}\n\n"
    "Responda **apenas** com base no CONTEXTO. Se não estiver no CONTEXTO, "
    "diga que não consta na base e sugira atualização do RAG."
)

def _format_context(docs: List[Document], max_chars: int = 4000) -> str:
    partes = []
    total = 0
    for i, d in enumerate(docs, 1):
        trecho = d.page_content.strip()
        fonte = d.metadata.get('source') if isinstance(d.metadata, dict) else None
        header = f'[{i}] Fonte: {fonte}\n' if fonte else f'[{i}]\n'
        bloco = header + trecho + '\n'
        if total + len(bloco) > max_chars:
            break
        partes.append(bloco)
        total += len(bloco)
    return '\n'.join(partes).strip()

def _tem_evidencia_suficiente(
    docs: List[Document],
    min_hits: int = 1,
    min_chars: int = 200,
    score_key: Optional[str] = 'score',
    min_score: float = 0.3,
) -> bool:
    if not docs or len(docs) < min_hits:
        return False
    texto_total = sum(len(getattr(d, 'page_content', '') or '') for d in docs)
    if score_key:
        try:
            scores = []
            for d in docs:
                meta = getattr(d, 'metadata', {}) or {}
                val = meta.get(score_key)
                if isinstance(val, (int, float)):
                    scores.append(float(val))
                elif isinstance(val, list) and val and isinstance(val[0], (int, float)):
                    scores.append(float(val[0]))
            if scores and max(scores) < min_score:
                return False
        except Exception:
            pass
    return texto_total >= min_chars or len(docs) >= max(min_hits, 2)

def responder_rag(
    pergunta: str,
    retriever: Any,
    llm: Any,
    router_chain: Optional[Any] = None,
    k: int = 4,
    min_hits: int = 1,
    min_chars: int = 200,
    score_key: Optional[str] = 'score',
    min_score: float = 0.3,
    system_prefix: str = system_instrucao_pt,
    user_template: str = user_template_pt,
) -> str:
    # Responde somente com base no RAG. Se não houver evidência suficiente,
    # retorna a mensagem padrão de ausência. Router_chain é ignorado aqui.
    try:
        if hasattr(retriever, 'get_relevant_documents'):
            docs = retriever.get_relevant_documents(pergunta)[:k]
        else:
            docs = retriever.invoke(pergunta)
            if isinstance(docs, list):
                docs = docs[:k]
            else:
                docs = [docs]
    except Exception as e:
        return f"{mensagem_falta_dado} (Falha no retriever: {e})"

    if not isinstance(docs, list):
        docs = list(docs) if docs else []

    if not _tem_evidencia_suficiente(docs, min_hits, min_chars, score_key, min_score):
        return mensagem_falta_dado

    contexto = _format_context(docs)
    user_msg = user_template.format(question=pergunta, context=contexto)

    try:
        final_prompt = f"{system_prefix}\n\n{user_msg}"
        resposta = llm.invoke(final_prompt)
        content = getattr(resposta, 'content', None) or getattr(getattr(resposta, 'message', None) or object(), 'content', None)
        if content:
            return content
        return str(resposta)
    except Exception as e:
        return f"{mensagem_falta_dado} (Falha no LLM: {e})"

# Funções de cadeia específicas, todas baseadas em responder_rag

def itinerary_chain(query: str) -> str:
    return responder_rag(pergunta=query, retriever=retriever, llm=llm)

def logistics_chain(query: str) -> str:
    return responder_rag(pergunta=query, retriever=retriever, llm=llm)

def localinfo_chain(query: str) -> str:
    return responder_rag(pergunta=query, retriever=retriever, llm=llm)

def translation_chain(query: str) -> str:
    return responder_rag(pergunta=query, retriever=retriever, llm=llm)

### Router (classificador de intenção)
O roteador decide entre as cadeias: `roteiro-viagem`, `logistica-transporte`, `info-local`, `traducao-idiomas`.

In [18]:
router_prompt = ChatPromptTemplate.from_template(
    """
    Você é um roteador de intenções. Dado o texto do usuário, responda com um JSON válido com as chaves:
    - route: uma das opções ["roteiro-viagem", "logistica-transporte", "info-local", "traducao-idiomas"]
    - reasoning: breve justificativa (1–2 frases)
    - normalized_query: reescreva a consulta de forma clara e completa

    Exemplos:
    - "roteiro cultural em Paris por 3 dias" -> route="roteiro-viagem"
    - "como chegar ao Coliseu?" -> route="logistica-transporte"
    - "horário do Louvre e preço" -> route="info-local"
    - "frases básicas em japonês" -> route="traducao-idiomas"

    Usuário: {query}
    """
)

def route_query(query: str) -> Dict[str, Any]:
    prompt = router_prompt.format(query=query)
    raw = (llm | StrOutputParser()).invoke(prompt)
    
    # tentativa robusta de parse
    try:
        # Captura primeiro bloco JSON
        m = re.search(r"\{[\s\S]*\}", raw)
        data = json.loads(m.group(0) if m else raw)
    except Exception:
        # fallback mínimo
        data = {"route": "info-local", "reasoning": "fallback", "normalized_query": query}
    return data

### Orquestração
Mapeia a rota => executa a cadeia correspondente.


In [19]:
route_to_chain = {
    "roteiro-viagem": itinerary_chain,
    "logistica-transporte": logistics_chain,
    "info-local": localinfo_chain,
    "traducao-idiomas": translation_chain,
}

def routerchain_invoke(query: str) -> Dict[str, Any]:
    decision = route_query(query)
    route = decision.get("route", "info-local")
    norm_q = decision.get("normalized_query", query)
    chain = route_to_chain.get(route, localinfo_chain)
    answer = chain(norm_q)
    return {
        "route": route,
        "normalized_query": norm_q,
        "answer": answer,
        "router_reasoning": decision.get("reasoning", ""),
    }


### Testes rápidos
Executa alguns exemplos.

In [21]:
tests = [
    "roteiro cultural na cidade de Paris por 3 dias",
    "qual o horário de funcionamento da Catedral de Notre-Dame?",
    "quais são os restaurantes indicado de Paris?",
    "frases úteis para utilizar em Paris",
]

for q in tests:
    print("\n"+"="*80)
    print("Pergunta:", q)
    result = routerchain_invoke(q)
    print("Rota:", result["route"])
    print("— Router:", result["router_reasoning"])
    print("\nResposta:\n", result["answer"][:1500], "..." if len(result["answer"])>1500 else "")


Pergunta: roteiro cultural na cidade de Paris por 3 dias
Rota: roteiro-viagem
— Router: A consulta menciona um 'roteiro cultural' e um período específico de tempo ('3 dias') em uma cidade específica ('Paris'), o que sugere que o usuário está procurando por um plano de viagem.

Resposta:
 Para um roteiro cultural na cidade de Paris para um período de 3 dias, podemos seguir o perfil cultural descrito no contexto:

Dia 1: Visita à Catedral de Notre-Dame e Sainte-Chapelle.
Dia 2: Torre Eiffel, Museu Orsay, Arco do Triunfo e caminhada pela Champs-Élysées.
Dia 3: Palácio de Versalhes ou viagem de um dia a Giverny (casa de Monet).

Essas informações estão presentes no contexto fornecido, especificamente na seção relacionada a Paris. Não há mais detalhes sobre outros roteiros culturais específicos para Paris além disso. Se mais informações forem necessárias, sugiro uma atualização do RAG para incluir mais detalhes sobre roteiros culturais em Paris. 

Pergunta: qual o horário de funcionamento 